# Catching Hackers
For this lab, we will be simulating the data we will work with using the [`login_attempt_simulator` package](https://github.com/fenago/login-attempt-simulator). The simulator needs to generate random numbers from various distributions. Here is an example of each of them:

In [ ]:
%matplotlib inline
from visual_aids import sim_viz

_ = sim_viz.show_distributions()

*Note: the Poisson distribution is discrete while the others are continuous. We use the Poisson distribution to model arrivals (users coming to login for this example). Discrete distributions have PMFs (probability mass functions) instead of PDFs.*

The `simulate.py` script can be run from the command line to run the simulation:

In [ ]:
!python3 simulate.py -h

We will simulate November 2018 using a seed of 0 and making the user base:

In [ ]:
!python3 simulate.py -ms 0 30 "2018-11-01"

## Setup

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

log = pd.read_csv('logs/log.csv', index_col='datetime', parse_dates=True)
attacks = pd.read_csv(
    'logs/attacks.csv',
    converters={'start': np.datetime64, 'end': np.datetime64}
) # make start and end columns datetimes but not the index

## Understanding the Data
The login attempts recorded from the website look like this:

In [ ]:
log.head()

The labeled data we have to research how to detect the attackers looks like this:

In [ ]:
attacks.head()

We can use `shape` to see the number of attacks and login attempts:

In [ ]:
attacks.shape, log.shape

What percentage of IP addresses were from attackers?

In [ ]:
attacks.source_ip.nunique() / log.source_ip.nunique()

## EDA
Can we find suspicious activity looking at hourly attempts to log in?

In [ ]:
# attempts over time
log.assign(attempts=1).attempts.resample('1H').sum()\
    .plot(figsize=(15, 5), title='hourly attempts')\
    .set(xlabel='datetime', ylabel='attempts')

How many attempts came from each IP address?

In [ ]:
log.source_ip.value_counts().describe()

What does the distribution of attempts per IP address look like?

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 5))
log.source_ip.value_counts().plot(kind='box', ax=axes[0]).set_ylabel('attempts')
log.source_ip.value_counts().plot(kind='hist', bins=50, ax=axes[1]).set_xlabel('attempts')
fig.suptitle('Attempts per IP Address')

What percent of the top IP addresses are hackers?

In [ ]:
num_hackers = attacks.source_ip.nunique()
log.source_ip.value_counts().index[:num_hackers]\
    .isin(attacks.source_ip).sum() / num_hackers

What does the average hourly attempts per IP address look like over the time period?

In [ ]:
# attempts per ip address
log.assign(attempts=1).groupby('source_ip').attempts\
    .resample('1H').sum().unstack().mean()\
    .plot(figsize=(15, 5), title='average hourly attempts per IP address')\
    .set_ylabel('average hourly attempts per IP address')

What percentage of the time was a hacker's attempt successful?

In [ ]:
log[log.source_ip.isin(attacks.source_ip)]\
    .success.value_counts(normalize=True)

What percentage of the time are valid users' attempts successful?

In [ ]:
log[~log.source_ip.isin(attacks.source_ip)]\
    .success.value_counts(normalize=True)

Why is each group failing?

In [ ]:
pd.crosstab(
    index=pd.Series(
        log.source_ip.isin(attacks.source_ip), name='is_hacker'
    ), columns=log.failure_reason
)

How many times does a user try to log in per hour? Valid users don't make many mistakes with their credentials, so if the hackers make many attempts with many users, we flag it.

In [ ]:
# attempts per user
log.assign(attempts=1).groupby('username').attempts\
    .resample('1H').sum().unstack().mean()\
    .plot(figsize=(15, 5), title='average hourly attempts per user')\
    .set_ylabel('average hourly attempts per user')

Calculate metrics per IP address

In [ ]:
pivot = log.pivot_table(
    values='success', index=log.source_ip, 
    columns=log.failure_reason.fillna('success'), 
    aggfunc='count', fill_value=0
)
pivot.insert(0, 'attempts', pivot.sum(axis=1))
pivot = pivot.sort_values('attempts', ascending=False).assign(
    success_rate=lambda x: x.success / x.attempts,
    error_rate=lambda x: 1 - x.success_rate
)
pivot.head()

Most successful IP addresses:

In [ ]:
pivot.sort_values('success_rate', ascending=False).head()

What looks out of place with the five-number summary?

In [ ]:
pivot.describe()

Are there IP addresses being used with many distinct usernames?

In [ ]:
log.groupby('source_ip').agg(dict(username='nunique'))\
    .username.value_counts().describe()

### Visual Anomaly Detection
We had a bimodal distribution of attempts per IP address &mdash; will we have two clusters when we plot successes vs. attempts by IP address?

In [ ]:
pivot.plot(
    kind='scatter', x='attempts', y='success', 
    title='successes vs. attempts by IP address', alpha=0.25
)

Perhaps we can draw a boundary between these groups:

In [ ]:
ax = pivot.plot(
    kind='scatter', x='attempts', y='success', 
    title='successes vs. attempts by IP address', alpha=0.25
)
plt.axvline(125, label='sample boundary', color='red', linestyle='--')
plt.legend(loc='lower right')

Since we are in the research phase, we have some labeled data, so we can see if our boundary was correct:

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 5))
for ax in axes:
    sns.scatterplot(
        y=pivot.success, x=pivot.attempts, 
        hue=pivot.assign(
            is_hacker=lambda x: x.index.isin(attacks.source_ip)
        ).is_hacker,
        ax=ax, alpha=0.5
    )
    for spine in ['top', 'right']:
        ax.spines[spine].set_visible(False)
axes[1].set_xscale('log')
plt.suptitle('successes vs. attempts by IP address')

Can a box plot show us outliers corresponding to the top right cluster in the previous scatter plots?

In [ ]:
pivot[['attempts', 'success']].plot(
    kind='box', subplots=True, figsize=(10, 3),
    title='stats per IP address'
)

## Rule-based Anomaly Detection
We want to find the IP addresses with excessive amounts of attempts with low success rates and those attempting to log in with more unique usernames than we would deem normal (anomalies).

In [ ]:
hourly_ip_logs = log.assign(
    failures=lambda x: np.invert(x.success)
).groupby('source_ip').resample('1H').agg(
    {'username': 'nunique', 'success': 'sum', 'failures': 'sum'}
).assign(
    attempts=lambda x: x.success + x.failures,
    success_rate=lambda x: x.success / x.attempts,
    failure_rate=lambda x: 1 - x.success_rate
).dropna().reset_index()

The data we will use for rule-based anomaly detection looks like this:

In [ ]:
hourly_ip_logs.head()

### Percent Difference from Threshold
A simple rule would be to check if values are a certain percentage or more different from some threshold. 

#### Bootstrapping
In our case, the threshold will be an hourly baseline of login activity. We could use bootstrapping to calculate the baseline with random sample of size 10 for each hour:

In [ ]:
def get_baselines(hourly_ip_logs, func, *args, **kwargs):
    """
    Calculate hourly bootstrapped statistic per column.
    
    Parameters:
        - hourly_ip_logs: Data to sample from.
        - func: Statistic to calculate.
        - args: Additional positional arguments for `func`
        - kwargs: Additional keyword arguments for `func`
    
    Returns:
        `pandas.DataFrame` of hourly bootstrapped statistics
    """
    if isinstance(func, str):
        func = getattr(pd.DataFrame, func)

    return hourly_ip_logs\
        .assign(hour=lambda x: x.datetime.dt.hour).groupby('hour')\
        .apply(
            lambda x: x.sample(10, random_state=0, replace=True)\
                .pipe(func, *args, **kwargs, numeric_only=True)
        )

When we run our function, we get a bootstrapped hourly average to use as a baseline:

In [ ]:
averages = get_baselines(hourly_ip_logs, 'mean')
averages.shape

This technique however doesn't guarantee we won't mix any of the hacker activity into our baseline calculations:

In [ ]:
averages.nlargest(6, 'failure_rate')

#### Trimming
We need to remove some of the outliers for better baselines. Let's write a function to trim values beyond a given quantile:

In [ ]:
def trim(x, quantile):
    """Remove rows with entries for the username, attempts, or failure_rate columns above a given quantile."""
    mask = ((x.username <= x.username.quantile(quantile))
        & (x.attempts <= x.attempts.quantile(quantile))
        & (x.failure_rate <= x.failure_rate.quantile(quantile)))
    return x[mask]

This function can be used when we run `apply()` after `groupby()`:

In [ ]:
trimmed_hourly_logs = hourly_ip_logs\
    .assign(hour=lambda x: x.datetime.dt.hour)\
    .groupby('hour').apply(lambda x: trim(x, 0.95))\
    .drop(columns='hour').reset_index().iloc[:,2:]

Our trimmed baseline doesn't have really large values anymore:

In [ ]:
averages = get_baselines(trimmed_hourly_logs, 'mean')
averages.iloc[[19, 23, 3, 11, 14, 16]]

Next, we need a function to determine when our thresholds are exceeded. Our threshold will be some percentage of the baseline:

In [ ]:
def pct_change_threshold(hourly_ip_logs, baselines, pcts=None):
    """
    Return flagged IP addresses based on thresholds.
    
    Parameters:
        - hourly_ip_logs: Aggregated hourly data per IP address.
        - baselines: Hourly baselines per column in data.
        - pcts: Dictionary of custom percentages per column for
                calculating upper bound thresholds (baseline * pct).
                If not provided, pct will be 1.
    
    Returns:
        `pandas.Series` containing the IP addresses flagged.
    """
    pcts = {} if not pcts else pcts

    return hourly_ip_logs.assign(
        hour=lambda x: x.datetime.dt.hour
    ).join(
        baselines, on='hour', rsuffix='_baseline'
    ).assign(
        too_many_users=lambda x: x.username_baseline \
            * pcts.get('username', 1) <= x.username,
        too_many_attempts=lambda x: x.attempts_baseline \
            * pcts.get('attempts', 1) <= x.attempts,
        high_failure_rate=lambda x: x.failure_rate_baseline \
            * pcts.get('failure_rate', 1) <= x.failure_rate
    ).query(
        'too_many_users and too_many_attempts and high_failure_rate'
    ).source_ip.drop_duplicates()

Let's see how many IP addresses get flagged for being 25% greater than the mean baselines for distinct usernames, number of attempts, and failure rate:

In [ ]:
pct_from_mean_ips = pct_change_threshold(
    hourly_ip_logs, averages, 
    {key: 1.25 for key in ['username', 'attempts', 'failure_rate']}
)
pct_from_mean_ips.nunique()

### Tukey Fence
Another strategy is to use the upper bound of the Tukey fence. The multiplier on the IQR is a parameter we will want to tune:

In [ ]:
def tukey_fence_test(trimmed_data, logs, k, pct=None):
    """
    See which IP addresses get flagged with a Tukey fence with
    multiplier k and optional percent differences.
    
    Parameters: 
        - trimmed_data: The data to use to calculate the baselines
        - logs: The data to test
        - k: The multiplier for the IQR
        - pct: Dictionary of percentages per column for use with `pct_change_threshold()`
        
    Returns:
        `pandas.Series` of flagged IP addresses
    """
    q3 = get_baselines(trimmed_data, 'quantile', .75).drop(columns=['hour'])
    q1 = get_baselines(trimmed_data, 'quantile', .25).drop(columns=['hour'])
    iqr = q3 - q1
    upper_bound = (q3 + k * iqr).reset_index()
    return pct_change_threshold(logs, upper_bound, pct)

Let's see what gets flagged using the Tukey fence upper bound using a multiplier of 3 for the IQR:

In [ ]:
tukey_fence_ips = tukey_fence_test(
    trimmed_hourly_logs, hourly_ip_logs, k=3
)

We once again use `nunique()` to see the number of items flagged:

In [ ]:
tukey_fence_ips.nunique()

### Z-score
We can use the Z-score to flag values beyond a certain number of standard deviations above the mean:

In [ ]:
def z_score_test(trimmed_data, logs, cutoff):
    """
    See which IP addresses get flagged with a Z-score greater than
    or equal to a cutoff value.
    
    Parameters: 
        - trimmed_data: The data to use to calculate the baselines
        - logs: The data to test
        - cutoff: Flag row when z_score >= cutoff
        
    Returns:
        `pandas.Series` of flagged IP addresses
    """
    std_dev = get_baselines(trimmed_data, 'std').drop(columns=['hour'])
    averages = get_baselines(trimmed_data, 'mean').drop(columns=['hour'])

    return logs.assign(
        hour=lambda x: x.datetime.dt.hour
    ).join(
        std_dev.join(
            averages, 
            lsuffix='_std', 
            rsuffix='_mean'
        ),
        on='hour'
    ).assign(
        too_many_users=lambda x: \
            (x.username - x.username_mean) / x.username_std >= cutoff,
        too_many_attempts=lambda x: \
            (x.attempts - x.attempts_mean) / x.attempts_std >= cutoff,
        high_failure_rate=lambda x: \
            (x.failure_rate - x.failure_rate_mean) / x.failure_rate_std >= cutoff
    ).query(
        'too_many_users and too_many_attempts and high_failure_rate'
    ).source_ip.drop_duplicates()

z_score_ips = z_score_test(trimmed_hourly_logs, hourly_ip_logs, 3)
z_score_ips.nunique()

### Evaluating Methods
This is a classification problem with 4 outcomes:

In [ ]:
from visual_aids import ml_viz
_ = ml_viz.confusion_matrix()

We can write a function to calculate the metrics of the confusion matrix:

In [ ]:
def evaluate(alerted_ips, attack_ips, log_ips):
    """
    Calculate true positives (TP), false positives (FP), 
    true negatives (TN), and false negatives (FN) for 
    IP addresses flagged as suspicious.
    
    Parameters:
        - alerted_ips: `pandas.Series` of flagged IP addresses
        - attack_ips: `pandas.Series` of attacker IP addresses
        - log_ips: `pandas.Series` of all IP addresses seen
    
    Returns:
        Tuple of form (TP, FP, TN, FN)
    """
    tp = alerted_ips.isin(attack_ips).sum()
    tn = np.invert(np.isin(log_ips[~log_ips.isin(alerted_ips)].unique(), attack_ips)).sum()
    fp = np.invert(alerted_ips.isin(attack_ips)).sum()
    fn = np.invert(attack_ips.isin(alerted_ips)).sum()
    return tp, fp, tn, fn

Using, partials we can reduce our typing later by providing the arguments only once:

In [ ]:
# make this easier to call
from functools import partial
scores = partial(evaluate, attack_ips=attacks.source_ip, log_ips=pivot.index)

#### Percent Difference from Mean
Let's see how well the percent difference from the mean method did. Using our partial, we get all the components of the confusion matrix:

In [ ]:
tp, fp, tn, fn = scores(pct_from_mean_ips)

We can use these to calculate the **false positive rate (FPR)** or the false alarm rate:

$$ FPR = \frac{FP}{FP + TN} $$ 

and the **false discovery rate (FDR)** or the percentage of our alarms that are wrong:

$$ FDR = \frac{FP}{FP + TP} $$ 

In [ ]:
# fpr, fdr
fp / (fp + tn), fp / (fp + tp)

To look into false negatives, we can calculate the **false negative rate (FNR)** or the miss rate:

$$ FNR = \frac{FN}{FN + TP} $$

and the **false omission rate (FOR)**:

$$ FOR = \frac{FN}{FN + TN} $$

In [ ]:
# fnr, for
fn / (fn + tp), fn / (fn + tn)

Let's create a function to calculate all of this for us, so we can compare the methods:

In [ ]:
def classification_stats(tp, fp, tn, fn):
    """Calculate metrics"""
    return {
        'FPR': fp / (fp + tn),
        'FDR': fp / (fp + tp),
        'FNR': fn / (fn + tp),
        'FOR': fn / (fn + tn)
    }

Percent difference from the mean using trimmed baselines performs well all around:

In [ ]:
classification_stats(tp, fp, tn, fn)

#### Percent Difference from the Median
We don't need to use the trimmed data here because the median is robust to outliers:

In [ ]:
medians = get_baselines(hourly_ip_logs, 'median')
pct_from_median_ips = pct_change_threshold(
    hourly_ip_logs, medians, 
    {key: 1.25 for key in ['username', 'attempts', 'failure_rate']}
)
tp, fp, tn, fn = scores(pct_from_median_ips)
classification_stats(tp, fp, tn, fn)

#### Comparing Methods
We can use a `DataFrame` object to easily compare the methods we tried. All perform well, which one we use in practice will depend on the cost of false negatives vs. false positives:

In [ ]:
pd.DataFrame({
    method: classification_stats(*scores(ips))
    for method, ips in {
        'means': pct_from_mean_ips,
        'medians': pct_from_median_ips,
        'Tukey fence': tukey_fence_ips,
        'Z-scores': z_score_ips
    }.items()
})

<hr>
<div style="overflow: hidden; margin-bottom: 10px;">
    <div style="float: left;">
         <a href="../lab_12/financial_analysis.ipynb">
            <button>&#8592; Lab 12</button>
        </a>
    </div>
    <div style="float: right;">
        <a href="../../solutions/lab_13/solutions.ipynb">
            <button>Solutions</button>
        </a>
        <a href="../ch_09/red_wine.ipynb">
            <button>Lab 9 &#8594;</button>
        </a>
    </div>
</div>
<hr>